In [173]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
from creation_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'data/N8_filt_10mino3.csv',index_col = 0)
print df_P[:5]

                       O3_ppb      UnixTime      e2v03  Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                     
2014-07-13 03:40:00        30  1.405223e+09  95.714286  23.6  56.885714   
2014-07-13 03:41:00        32  1.405223e+09  95.857143  23.6  56.757143   
2014-07-13 03:42:00        33  1.405223e+09  93.285714  23.6  56.871429   
2014-07-13 03:43:00        32  1.405223e+09  93.142857  23.6  56.957143   
2014-07-13 03:44:00        28  1.405223e+09  96.714286  23.6  57.042857   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  
DATE (MM/DD/YYYY)_MST                                                          
2014-07-13 03:40:00                 100.86232                0             30  
2014-07-13 03:41:00                 100.71634                0             32  
2014-07-13 03:42:00                 100.56996                0             33  
2014-07-13 03:43:00                 100.42319                0            

In [175]:
df_P.drop(df_P.columns[[5]], axis=1, inplace=True)
print df_P[:5]

                       O3_ppb      UnixTime      e2v03  Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                     
2014-07-13 03:40:00        30  1.405223e+09  95.714286  23.6  56.885714   
2014-07-13 03:41:00        32  1.405223e+09  95.857143  23.6  56.757143   
2014-07-13 03:42:00        33  1.405223e+09  93.285714  23.6  56.871429   
2014-07-13 03:43:00        32  1.405223e+09  93.142857  23.6  56.957143   
2014-07-13 03:44:00        28  1.405223e+09  96.714286  23.6  57.042857   

                       days from start  ref_o3_smooth  
DATE (MM/DD/YYYY)_MST                                  
2014-07-13 03:40:00                  0             30  
2014-07-13 03:41:00                  0             32  
2014-07-13 03:42:00                  0             33  
2014-07-13 03:43:00                  0             32  
2014-07-13 03:44:00                  0             28  


In [176]:
#ozone terms
df_P['O3_sqrt'] = np.sqrt(df_P['e2v03'].astype(float))
df_P['O3_sq'] = df_P['e2v03']**2
df_P['O3_cu'] = df_P['e2v03']**3
df_P['ln_O3'] = np.log(df_P['e2v03'])

#temp terms
df_P['temp_sqrt'] = np.sqrt(df_P['Temp'].astype(float))
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] =np.log(df_P['Temp'])

#rh terms
df_P['rh_sqrt'] = np.sqrt(df_P['Rh'].astype(float))
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])

#ozone/temp interactions
df_P['03_mult_temp'] = df_P['e2v03']*df_P['Temp']
df_P['03_sq_mult_temp_sq'] = ((df_P['e2v03'])**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['e2v03'])

#ozone/rh interactions
df_P['03_mult_rh'] = df_P['e2v03']*df_P['Rh']
df_P['03_sq_mult_rh_sq'] = (df_P['e2v03']**2)*df_P['Rh']**2
df_P['ln_03_mult_ln_rh'] = np.log(df_P['Rh'])*np.log(df_P['e2v03'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_sq_mult_rh_sq'] = (df_P['Temp']**2)*df_P['Rh']**2
df_P['ln_temp_mult_ln_rh'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])

#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['e2v03']*df_P['Rh']*df_P['Temp']
df_P['03_sq_mult_rh_sq_&_temp_sq'] = (df_P['e2v03']**2)*(df_P['Rh']**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_rh_&_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])*np.log(df_P['e2v03'])

feature_list = ['e2v03', 'ln_O3', 'Temp', 'ln_temp', 'Rh', 'ln_rh']

for i in range(0,len(feature_list)):
    make_func_caller_find_lag_integral(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lag_integral(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 5, 5, 15, feature_list[i])

features = list(df_P.columns)
print features

['O3_ppb', 'UnixTime', 'e2v03', 'Temp', 'Rh', 'days from start', 'ref_o3_smooth', 'O3_sqrt', 'O3_sq', 'O3_cu', 'ln_O3', 'temp_sqrt', 'temp_sq', 'temp_cu', 'ln_temp', 'rh_sqrt', 'rh_sq', 'rh_cu', 'ln_rh', '03_mult_temp', '03_sq_mult_temp_sq', 'ln_03_mult_ln_temp', '03_mult_rh', '03_sq_mult_rh_sq', 'ln_03_mult_ln_rh', 'temp_mult_rh', 'temp_sq_mult_rh_sq', 'ln_temp_mult_ln_rh', '03_mult_rh_&_temp', '03_sq_mult_rh_sq_&_temp_sq', 'ln_03_mult_ln_rh_&_ln_temp', 'e2v03_int_lag_15', 'e2v03_int_lag_30', 'e2v03_int_lag_45', 'e2v03_int_lag_60', 'e2v03_int_lag_75', 'e2v03_int_lag_90', 'e2v03_int_lead_15', 'e2v03_int_lead_30', 'e2v03_int_lead_45', 'e2v03_int_lead_60', 'e2v03_int_lead_75', 'e2v03_int_lead_90', 'diff', 'e2v03_slope_lag_15', 'e2v03_slope_lag_30', 'e2v03_slope_lag_45', 'e2v03_slope_lag_60', 'e2v03_slope_lag_75', 'e2v03_slope_lag_90', 'e2v03_slope_lead_15', 'e2v03_slope_lead_30', 'e2v03_slope_lead_45', 'e2v03_slope_lead_60', 'e2v03_slope_lead_75', 'e2v03_slope_lead_90', 'e2v03_int_lag_5'

In [177]:
#Delete the first and last 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[90:len(df_P['e2v03'])-90]

In [178]:
#ozone/temp
df_P['o3_temp_int_lag_30'] = df_P['e2v03_int_lag_30'] * df_P['Temp_int_lag_30'] 
df_P['o3_temp_slope_lag_30'] = df_P['e2v03_slope_lag_30'] * df_P['Temp_slope_lag_30']
df_P['o3_temp_int_lag_60'] = df_P['e2v03_int_lag_60'] * df_P['Temp_int_lag_60'] 
df_P['o3_temp_slope_lag_60'] = df_P['e2v03_slope_lag_60'] * df_P['Temp_slope_lag_60'] 
df_P['o3_temp_int_lag_90'] = df_P['e2v03_int_lag_90'] * df_P['Temp_int_lag_90'] 
df_P['o3_temp_slope_lag_90'] = df_P['e2v03_slope_lag_90'] * df_P['Temp_slope_lag_90']
df_P['o3_temp_int_lag_5'] = df_P['e2v03_int_lag_5'] * df_P['Temp_int_lag_5'] 
df_P['o3_temp_slope_lag_5'] = df_P['e2v03_slope_lag_5'] * df_P['Temp_slope_lag_5']

df_P['ln_o3_temp_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_temp_int_lag_30'] 
df_P['ln_o3_temp_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_temp_slope_lag_30']
df_P['ln_o3_temp_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_temp_int_lag_60'] 
df_P['ln_o3_temp_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_temp_slope_lag_60'] 
df_P['ln_o3_temp_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_temp_int_lag_90'] 
df_P['ln_o3_temp_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_temp_slope_lag_90']
df_P['ln_o3_temp_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_temp_int_lag_5'] 
df_P['ln_o3_temp_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_temp_slope_lag_5']

df_P['o3_temp_int_lead_30'] = df_P['e2v03_int_lead_30'] * df_P['Temp_int_lead_30'] 
df_P['o3_temp_slope_lead_30'] = df_P['e2v03_slope_lead_30'] * df_P['Temp_slope_lead_30'] 
df_P['o3_temp_int_lead_60'] = df_P['e2v03_int_lead_60'] * df_P['Temp_int_lead_60'] 
df_P['o3_temp_slope_lead_60'] = df_P['e2v03_slope_lead_60'] * df_P['Temp_slope_lead_60'] 
df_P['o3_temp_int_lead_90'] = df_P['e2v03_int_lead_90'] * df_P['Temp_int_lead_90'] 
df_P['o3_temp_slope_lead_90'] = df_P['e2v03_slope_lead_90'] * df_P['Temp_slope_lead_90']
df_P['o3_temp_int_lead_5'] = df_P['e2v03_int_lead_5'] * df_P['Temp_int_lead_5'] 
df_P['o3_temp_slope_lead_5'] = df_P['e2v03_slope_lead_5'] * df_P['Temp_slope_lead_5']

df_P['ln_o3_temp_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_temp_int_lead_30'] 
df_P['ln_o3_temp_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_temp_slope_lead_30'] 
df_P['ln_o3_temp_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_temp_int_lead_60'] 
df_P['ln_o3_temp_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_temp_slope_lead_60'] 
df_P['ln_o3_temp_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_temp_int_lead_90'] 
df_P['ln_o3_temp_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_temp_slope_lead_90']
df_P['ln_o3_temp_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_temp_int_lead_5'] 
df_P['ln_o3_temp_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_temp_slope_lead_5']

#temp/rh
df_P['temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5']

df_P['ln_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5']

df_P['temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']
df_P['ln_temp_rh_int_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/rh
df_P['o3_rh_int_lag_30'] = df_P['e2v03_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['o3_rh_slope_lag_30'] = df_P['e2v03_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['o3_rh_int_lag_60'] = df_P['e2v03_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['o3_rh_slope_lag_60'] = df_P['e2v03_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['o3_rh_int_lag_90'] = df_P['e2v03_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['o3_rh_slope_lag_90'] = df_P['e2v03_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['o3_rh_int_lag_5'] = df_P['e2v03_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['o3_rh_slope_lag_5'] = df_P['e2v03_slope_lag_5'] * df_P['Rh_slope_lag_5'] 

df_P['ln_o3_rh_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_o3_rh_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_o3_rh_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_o3_rh_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_o3_rh_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_o3_rh_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_o3_rh_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_o3_rh_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] 

df_P['o3_rh_int_lead_30'] = df_P['e2v03_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['o3_rh_slope_lead_30'] = df_P['e2v03_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['o3_rh_int_lead_60'] = df_P['e2v03_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['o3_rh_slope_lead_60'] = df_P['e2v03_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['o3_rh_int_lead_90'] = df_P['e2v03_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['o3_rh_slope_lead_90'] = df_P['e2v03_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['o3_rh_int_lead_5'] = df_P['e2v03_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['o3_rh_slope_lead_5'] = df_P['e2v03_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_o3_rh_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_o3_rh_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_o3_rh_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_o3_rh_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_o3_rh_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_o3_rh_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_rh_slope_lead_90']
df_P['ln_o3_rh_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_rh_int_lead_5'] 
df_P['ln_o3_rh_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/temp/rh
df_P['o3_temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30']*df_P['e2v03_int_lag_30'] 
df_P['o3_temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] * df_P['e2v03_slope_lag_30'] 
df_P['o3_temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60']*df_P['e2v03_int_lag_60'] 
df_P['o3_temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] * df_P['e2v03_slope_lag_60'] 
df_P['o3_temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] * df_P['e2v03_int_lag_90']
df_P['o3_temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] * df_P['e2v03_slope_lag_90'] 
df_P['o3_temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] * df_P['e2v03_int_lag_5']
df_P['o3_temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5'] * df_P['e2v03_slope_lag_5'] 

df_P['ln_o3_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30']*df_P['ln_O3_int_lag_30'] 
df_P['ln_o3_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] * df_P['ln_O3_slope_lag_30'] 
df_P['ln_o3_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60']*df_P['ln_O3_int_lag_60'] 
df_P['ln_o3_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] * df_P['ln_O3_slope_lag_60'] 
df_P['ln_o3_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] * df_P['ln_O3_int_lag_90']
df_P['ln_o3_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] * df_P['ln_O3_slope_lag_90'] 
df_P['ln_o3_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] * df_P['ln_O3_int_lag_5']
df_P['ln_o3_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] * df_P['ln_O3_slope_lag_5'] 

df_P['o3_temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] * df_P['e2v03_int_lead_30'] 
df_P['o3_temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] * df_P['e2v03_slope_lead_30']
df_P['o3_temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] * df_P['e2v03_int_lead_60'] 
df_P['o3_temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] * df_P['e2v03_slope_lead_60'] 
df_P['o3_temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] * df_P['e2v03_int_lead_90']
df_P['o3_temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90'] * df_P['e2v03_slope_lead_90']
df_P['o3_temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] * df_P['e2v03_int_lead_5']
df_P['o3_temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5'] * df_P['e2v03_slope_lead_5']

df_P['ln_o3_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] * df_P['ln_O3_int_lead_30'] 
df_P['ln_o3_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] * df_P['ln_O3_slope_lead_30']
df_P['ln_o3_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] * df_P['ln_O3_int_lead_60'] 
df_P['ln_o3_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] * df_P['ln_O3_slope_lead_60']
df_P['ln_o3_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] * df_P['ln_O3_int_lead_90']
df_P['ln_o3_temp_rh_slope_lead_90'] = df_P['ln_temp_slope_lead_90'] * df_P['ln_rh_slope_lead_90'] * df_P['ln_O3_slope_lead_90']
df_P['ln_o3_temp_rh_int_lead_5'] = df_P['ln_temp_int_lead_5'] * df_P['ln_rh_int_lead_5'] * df_P['ln_O3_int_lead_5']
df_P['ln_o3_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5'] * df_P['ln_O3_slope_lead_5']

In [179]:
df_P.to_csv(path_or_buf = 'data/N8_raw_features.csv')